In [11]:
avg_rating_csv = "hdfs:///user/cloudera/data/input/avg_rating.csv"
input_meta = "hdfs:///user/cloudera/data/input/meta_Electronics.json"  
output = "hdfs:///user/cloudera/data/output_spark_prod_title"

In [12]:
def get_dict_from_csv(csv_row):
    result = list(csv.reader([csv_row]))
    return (result[0][0], result[0][1])

In [13]:
import ast

def get_prod_id_and_title(product_json_item):
    dict_product_item = ast.literal_eval(product_json_item)
    return (dict_product_item['asin'], dict_product_item.get('title', None))

In [14]:
import csv, io

def list_to_csv_str(x):
    """Given a list of strings, returns a properly-csv-formatted string."""
    output = io.StringIO("")
    csv.writer(output).writerow(x)
    return output.getvalue().strip() # remove extra newline

In [15]:
rdd_avg_ratings = sc.textFile(avg_rating_csv).map(lambda row: get_dict_from_csv(row))

In [16]:
rdd_titles = sc.textFile(input_meta).map(lambda row: get_prod_id_and_title(row)).filter(lambda x: x[1] is not None)

In [17]:
rdd_id_title_rating = rdd_avg_ratings.join(rdd_titles).map(lambda x: (x[0], "\"" + x[1][1] + "\"", x[1][0]))

In [18]:
rdd_id_title_rating = rdd_id_title_rating.filter(lambda x: x[0] is not None and x[1] is not None and x[2] is not None)

In [20]:
rdd_id_title_rating.map(list_to_csv_str).saveAsTextFile(output)